# Web Scraping - extraction des données

In [ ]:
# Importation des librairies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Définition d'une fonction qui scrape toutes les pages d'une entreprise sur TrustPilot
def scraping_commentaires(url):

    # Récupération du contenu de la page de l' "url" passée en argument de la fonction
    reponse = requests.get(url)
    if reponse.status_code == 200:
        
        # Création de l'objet BeautifulSoup pour analyser le code HTML
        soup = BeautifulSoup(reponse.text, 'lxml')
        nbre_de_pages = soup.find('a', attrs = {'name' : 'pagination-button-next'}).find_previous().text.strip()

        nom_pseudo = []
        date_publication = []
        note = []
        titre = []
        avis = []
        date_experience = []
        nbre_commentaires = []
        pays = []

        # Boucle sur les pages
        for page in range(1, int(nbre_de_pages) + 1):

            # Récupération du contenu de n-ième page
            reponse = requests.get(url + '?page=' + str(page))
            if reponse.status_code == 200:

                # Création de l'objet BeautifulSoup pour analyser le code HTML
                soup = BeautifulSoup(reponse.text, 'lxml')
                commentaires = soup.find_all('div', class_='styles_reviewCardInner__EwDq2')
            
                # Boucle sur les commentaires d'une page
                for commentaire in commentaires:
        
                    # Nom / Pseudo du client
                    nom_pseudo_element = commentaire.find('span', class_='typography_heading-xxs__QKBS8 typography_appearance-default__AAY17')
                    nom_pseudo.append(nom_pseudo_element.text.strip() if nom_pseudo_element else '')

                    # Date de l'avis du client
                    date_publication_element = commentaire.find('time')['datetime']
                    date_publication.append(date_publication_element.strip() if date_publication_element else '')

                    # Note liée à l'avis du client
                    note_element = commentaire.find('div', class_='styles_reviewHeader__iU9Px')['data-service-review-rating']
                    note.append(note_element.strip() if note_element else '')
                 
                    # Titre de l'avis du client
                    titre_element = commentaire.find('h2', class_='typography_heading-s__f7029 typography_appearance-default__AAY17')
                    titre.append(titre_element.text.strip() if titre_element else '')
                
                    # Contenu de l'avis du client
                    avis_element = commentaire.find('p', class_='typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn')
                    avis.append(avis_element.text.strip() if avis_element else '')

                    # Date de l'expérience du client
                    date_experience_element = commentaire.find('p', class_='typography_body-m__xgxZ_ typography_appearance-default__AAY17')
                    date_experience.append(date_experience_element.text.strip() if date_experience_element else '')
                    
                    # Nombre d'avis du client
                    nbre_commentaires_element = commentaire.find('span', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l')
                    nbre_commentaires.append(nbre_commentaires_element.text.strip() if nbre_commentaires_element else '')
                    
                    # Pays du client
                    pays_element = commentaire.find('div', class_='typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua')
                    pays.append(pays_element.text.strip() if pays_element else '')
                    
            else:
                print('Erreur lors de la récupération de la page', page, ':', reponse.status_code)

        df = pd.DataFrame({'Nom / Pseudo' : nom_pseudo,
                           'Date de publication' : date_publication,
                           'Note' : note,
                           'Titre' : titre,
                           'Avis' : avis,
                           'Date de l\'expérience' : date_experience,
                           'Nombre de commentaires' : nbre_commentaires,
                           'Pays' : pays})
        match = re.search(r'/review/(?:www\.)?(.*?)\.', url)
        df['Banque'] = match.group(1)
        df.to_csv('../csv/scraping_commentaires_' + match.group(1) + '.csv', index = False)
        return df
  
    else:
        print('Erreur lors de la récupération de la 1ère page :', reponse.status_code)


In [ ]:
scraping_commentaires("https://fr.trustpilot.com/review/hellobank.fr")

In [ ]:
df_boursobank = pd.read_csv('scraping_commentaires_boursobank.csv')
df_boursobank.shape

In [ ]:
df_fortuneo = pd.read_csv('scraping_commentaires_fortuneo.csv')
df_fortuneo.shape

In [ ]:
df_monabanq = pd.read_csv('scraping_commentaires_monabanq.csv')
df_monabanq.shape

In [ ]:
df_hellobank = pd.read_csv('scraping_commentaires_hellobank.csv')
df_hellobank.shape

In [ ]:
df_total = pd.concat(objs = [df_boursobank, df_fortuneo, df_monabanq, df_hellobank], axis = 0, ignore_index = True)
df_total.shape

In [ ]:
df_total.to_csv('../csv/scraping_commentaires_4_banques.csv', index = False)

In [ ]:
#Code suplementaire pour enregister les données 
# Enregistrer le DataFrame dans un fichier CSV
df.to_csv('../csv/comments_data_boursobank.csv', index=False)